# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId'] , axis=1)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 計算df整體相關係數, 並繪製成熱圖
import seaborn as sns
import matplotlib.pyplot as plt
corr = df.corr()
sns.heatmap(corr)
plt.show()

<Figure size 640x480 with 2 Axes>

In [3]:
abs(corr)

,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000000,0.338481,0.077221,0.035322,0.081629,0.257307
Pclass,0.338481,1.000000,0.369226,0.083081,0.018443,0.549500
Age,0.077221,0.369226,1.000000,0.308247,0.189119,0.096067
SibSp,0.035322,0.083081,0.308247,1.000000,0.414838,0.159651
Parch,0.081629,0.018443,0.189119,0.414838,1.000000,0.216225
Fare,0.257307,0.549500,0.096067,0.159651,0.216225,1.000000


In [4]:
# 記得刪除 Survived
df = df.drop(['Survived'] , axis=1)

#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 鐵達尼生存率預測中，試著變更兩種以上的相關係數門檻值，觀察預測能力是否提升?
## 原始 0.704
    利用 higher correlation selection 後，0.679, 0.683, 沒有進步
    嘗試比較 GBRegression 發現，整個爛掉

In [5]:
# 原始特徵 + 邏輯斯迴歸
import warnings
warnings.filterwarnings('ignore')
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [6]:
# 篩選相關係數1
"""
Your Code Here
"""

# 篩選相關係數大於 0.1 或小於 -0.1 的特徵
high_list = list(corr[(corr['Survived']>0.1) | (corr['Survived']<-0.1)].index)
high_list.pop(0)
print(high_list)

['Pclass', 'Fare']


In [7]:
# 特徵1 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df[high_list])
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6791567235397566

In [8]:
# 篩選相關係數2
"""
Your Code Here
"""
high_list = list(corr[(corr['Survived']>0.08) | (corr['Survived']<-0.08)].index)
high_list.pop(0)
print(high_list)

['Pclass', 'Parch', 'Fare']


In [9]:
# 特徵2 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df[high_list])
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6836511764895717

In [10]:
# 原始特徵 + GBRegression
from sklearn.ensemble import GradientBoostingRegressor
X_train=MMEncoder.fit_transform(df)
model = GradientBoostingRegressor()
cross_val_score(model, X_train, train_Y, cv=5).mean()
# 爛到爆 ~~

0.18291536739414999

In [11]:
X_train=MMEncoder.fit_transform(df[high_list])
model = GradientBoostingRegressor()
cross_val_score(model, X_train, train_Y, cv=5).mean()

0.1597373685662835

# 作業2
* 續上題，使用 L1 Embedding 做特徵選擇(自訂門檻)，觀察預測能力是否提升?
# 原始 0.704
    利用 L1 幾個 Alpha 調參後，沒有進步 [0.01, 0.005, 0.001]。約0.67 ~ 0.685 之間
    也許可以試看看grid search，但是根據 [0.01, 0.005, 0.001]觀察，猜想不太能有進展。

In [12]:
from sklearn.linear_model import Lasso
"""
Your Code Here, select parameter alpha 
"""
from sklearn.linear_model import Lasso
L1_Reg = Lasso(alpha=0.001)
train_X = MMEncoder.fit_transform(df[high_list])
L1_Reg.fit(train_X, train_Y)
L1_Reg.coef_

array([-0.35355706,  0.21641813,  0.28227934])

In [13]:
from itertools import compress
L1_mask = list((L1_Reg.coef_>0) | (L1_Reg.coef_<0))
L1_list = list(compress(list(df), list(L1_mask)))
L1_list

['Pclass', 'Age', 'SibSp']

In [14]:
# L1_Embedding 特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df[L1_list])
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6859237594896352

In [15]:
# alpha = 0.005
L1_Reg = Lasso(alpha=0.005)
train_X = MMEncoder.fit_transform(df[high_list])
L1_Reg.fit(train_X, train_Y)
L1_mask = list((L1_Reg.coef_>0) | (L1_Reg.coef_<0))
L1_list = list(compress(list(df), list(L1_mask)))
L1_list
train_X = MMEncoder.fit_transform(df[L1_list])
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6847938159868103

In [16]:
# alpha = 0.01
L1_Reg = Lasso(alpha=0.01)
train_X = MMEncoder.fit_transform(df[high_list])
L1_Reg.fit(train_X, train_Y)
L1_mask = list((L1_Reg.coef_>0) | (L1_Reg.coef_<0))
L1_list = list(compress(list(df), list(L1_mask)))
L1_list
train_X = MMEncoder.fit_transform(df[L1_list])
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6791567235397566

In [17]:
# alpha = 0.0005
L1_Reg = Lasso(alpha=0.0005)
train_X = MMEncoder.fit_transform(df[high_list])
L1_Reg.fit(train_X, train_Y)
L1_mask = list((L1_Reg.coef_>0) | (L1_Reg.coef_<0))
L1_list = list(compress(list(df), list(L1_mask)))
L1_list
train_X = MMEncoder.fit_transform(df[L1_list])
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6859237594896352